In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install -q diffusers transformers accelerate xformers bitsandbytes gradio huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
from zipfile import ZipFile
from PIL import Image

In [ ]:
instance_dir = "/content/dataset/instance"
os.makedirs(instance_dir, exist_ok=True)

In [ ]:
with ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("temp_vangogh")

In [ ]:
for i, file in enumerate(os.listdir("temp_vangogh")):
    if file.lower().endswith((".jpg", ".png")):
        img = Image.open(os.path.join("temp_vangogh", file)).convert("RGB")
        img = img.resize((512, 512))
        img.save(os.path.join(instance_dir, f"vangogh_{i}.jpg"))

print(f"Saved {i+1} images to {instance_dir}")

In [ ]:
!accelerate config default

!accelerate launch \
  /usr/local/lib/python3.10/dist-packages/diffusers/commands/train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="{instance_dir}" \
  --output_dir="/content/vangogh_model" \
  --instance_prompt="a painting in vangogh style" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --max_train_steps=800 \
  --mixed_precision="fp16" \
  --use_8bit_adam

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("/content/vangogh_model", torch_dtype=torch.float16).to("cuda")

prompt = "a fantasy landscape in vangogh style"
image = pipe(prompt).images[0]
image.save("/content/generated_vangogh.png")
image.show()

In [ ]:
import gradio as gr

def generate(prompt):
    return pipe(prompt).images[0]

gr.Interface(fn=generate, inputs="text", outputs="image", title="Van Gogh Style Generator").launch()